In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'd:\\MLOps-Project\\Kidney-disease-classification-mlops'

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    

In [12]:
from cnnClassifier.utils.common import read_yaml, create_directories,save_json
from cnnClassifier.constant import *

from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constant import *
# Configuration
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.atifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model='artifacts/training/model.h5',
            training_data='artifacts/data_ingestion/unzip/kidney-ct-scan-image',
            mlflow_uri='https://dagshub.com/azizulhakim8291/Kidney-disease-classification-mlops.mlflow',
            all_params= self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    
    

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [17]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
        

In [15]:
import dagshub
dagshub.init(repo_owner='azizulhakim8291', repo_name='Kidney-disease-classification-mlops', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2024-08-01 23:40:02,445: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/azizulhakim8291/Kidney-disease-classification-mlops "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "azizulhakim8291/Kidney-disease-classification-mlops"

[2024-08-01 23:40:02,453: INFO: helpers: Initialized MLflow to track repo "azizulhakim8291/Kidney-disease-classification-mlops"]


Repository azizulhakim8291/Kidney-disease-classification-mlops initialized!

[2024-08-01 23:40:02,458: INFO: helpers: Repository azizulhakim8291/Kidney-disease-classification-mlops initialized!]


In [19]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-01 23:44:03,754: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-01 23:44:03,764: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-01 23:44:03,770: INFO: common: Created directory  at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 13s 1s/step - loss: 0.5003 - accuracy: 0.9568
[2024-08-01 23:44:17,498: INFO: common: Json file saved at: scores.json]


2024/08/01 23:44:20 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-01 23:44:22,432: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp8n0wc3k0\model\data\model\assets
[2024-08-01 23:44:24,256: INFO: builder_impl: Assets written to: C:\Users\User\AppData\Local\Temp\tmp8n0wc3k0\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/08/01 23:47:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
